In [1]:
import pandas as pd
import numpy as np

## set display options for viewing result

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## read movie information

In [3]:
movies_df = pd.read_csv('../dataset/title.basics.tsv',header=0, index_col=None, sep='\t', dtype=str)

# replace NaN with None
movies_df.replace({np.nan:None}, inplace=True)

## read ratings information

In [4]:
# cast numeric data to appropriate data type as we will be filtering based on these numeric fields:

ratings_df = pd.read_csv('../dataset/title.ratings.tsv', header=0,index_col=None,sep='\t',converters={'tconst': str, 'averageRating': float, 'numVotes': int})

# replace NaN with None
ratings_df.replace({np.nan:None}, inplace=True)

## clean column name and data

In [5]:
movies_df.columns = [col.lower() for col in movies_df.columns]
ratings_df.columns = [col.lower() for col in ratings_df.columns]

## print data types of both dataframes

In [6]:
movies_df.dtypes

tconst            object
titletype         object
primarytitle      object
originaltitle     object
isadult           object
startyear         object
endyear           object
runtimeminutes    object
genres            object
dtype: object

In [7]:
ratings_df.dtypes

tconst            object
averagerating    float64
numvotes           int64
dtype: object

## merge dataframes to analyze movies data

In [8]:
merged_df = movies_df.merge(ratings_df, how='left', on='tconst')

In [9]:
# getting movies released after 2010 with avg rating above 7 and number of votes > 5000:
req_movies_df = merged_df[(merged_df['numvotes'] > 5000) & (merged_df['averagerating'] > 7.0) & (merged_df['startyear'] > '2010') & (merged_df['titletype']=='movie')][['primarytitle', 'isadult', 'startyear', 'runtimeminutes', 'genres', 'averagerating', 'numvotes']]

In [11]:
# set appropriate datatype for req_movies_df:

col_dtypes = {
    'primarytitle': str,
    'isadult': int,
    'startyear': int,
    'runtimeminutes': int,
    'genres': str,
    'averagerating': float,
    'numvotes': int
}

for col, dtype in col_dtypes.items():
    req_movies_df[col] = req_movies_df[col].astype(dtype)

## view the req_movies_df

In [12]:
req_movies_df.head(4)

,primarytitle,isadult,startyear,runtimeminutes,genres,averagerating,numvotes
302127,Wazir,0,2016,103,"Action,Crime,Drama",7.1,19226
344878,The Secret Life of Walter Mitty,0,2013,114,"Adventure,Comedy,Drama",7.3,319087
419478,Alita: Battle Angel,0,2019,122,"Action,Adventure,Sci-Fi",7.3,269169
425523,Lincoln,0,2012,150,"Biography,Drama,History",7.3,261742


## get streaming info from imdb datasets

In [31]:
streaming_availability = pd.read_csv('../dataset/MoviesOnStreamingPlatforms_updated.csv', header=0, index_col=None, converters={'Title': str, 'Year': int, 'Rotten Tomatoes': str, 'Netflix': int, 'Prime Video': int, 'Hulu': int, 'Disney+': int, 'Language': str})
streaming_availability.replace({np.nan:None}, inplace=True)

In [32]:
# clean column names to uniform lowercase:

streaming_availability.columns = [col.lower() for col in streaming_availability.columns]

## enrich req_movies_df with streaming information with available data

In [36]:
# convert title to lowercase to use as join key:

req_movies_df['primarytitle'] = req_movies_df['primarytitle'].apply(lambda x: x.lower())
streaming_availability['title'] = streaming_availability['title'].apply(lambda x: x.lower())

## merge req_movies_df with streaming availability info

In [37]:
enriched_movie_df = req_movies_df.merge(streaming_availability, left_on='primarytitle', right_on='title', how='left')

In [38]:
enriched_movie_df.replace({np.nan:None}, inplace=True)

,primarytitle,isadult,startyear,runtimeminutes,genres_x,averagerating,numvotes,id,title,year,age,imdb,rotten tomatoes,netflix,hulu,prime video,disney+,type,directors,genres_y,country,language,runtime
0,wazir,0,2016,103,"Action,Crime,Drama",7.1,19226,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,the secret life of walter mitty,0,2013,114,"Adventure,Comedy,Drama",7.3,319087,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,alita: battle angel,0,2019,122,"Action,Adventure,Sci-Fi",7.3,269169,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## add streaming info column `streaming_on`

In [40]:
enriched_movie_df['streaming_on'] = np.where(enriched_movie_df['netflix']==1, 'Netflix',
                                    np.where(enriched_movie_df['hulu']==1, 'Hulu',
                                    np.where(enriched_movie_df['prime video']==1, 'Prime',
                                    np.where(enriched_movie_df['disney+']==1, 'Disney+', 'data_unavailable'))))

In [42]:
final_enriched_movies_df = enriched_movie_df[['primarytitle', 'isadult', 'language', 'startyear', 'runtimeminutes', 'genres_x', 'averagerating', 'rotten tomatoes', 'numvotes', 'streaming_on']]

## rename final output columns:

In [43]:
final_enriched_movies_df.columns = ['title', 'is_adult', 'language', 'year', 'runtime_mins', 'genre', 'avg_rating', 'rotten tomatoes', 'num_votes', 'streaming_on']

In [44]:
# view final dataframe:

final_enriched_movies_df.head(4)

,title,is_adult,language,year,runtime_mins,genre,avg_rating,rotten tomatoes,num_votes,streaming_on
0,wazir,0,None,2016,103,"Action,Crime,Drama",7.1,None,19226,data_unavailable
1,the secret life of walter mitty,0,None,2013,114,"Adventure,Comedy,Drama",7.3,None,319087,data_unavailable
2,alita: battle angel,0,None,2019,122,"Action,Adventure,Sci-Fi",7.3,None,269169,data_unavailable
3,lincoln,0,None,2012,150,"Biography,Drama,History",7.3,None,261742,data_unavailable


## write the data to excel sorted by `title, year, rating and num_votes`

In [45]:
# sorts title asc, year desc, avg_rating desc and num_votes desc. the change is done inplace and ignoring index of dataframe:
final_enriched_movies_df.sort_values(by=['title', 'year', 'avg_rating', 'num_votes'], ascending=[True, False, False, False], ignore_index=True, inplace=True)

# write the sorted output to excel file:
final_enriched_movies_df.to_excel('./output/imdb_movies_2011_2022.xlsx', header=True, index=None, sheet_name='movies')

/var/folders/9m/b8mn27b56vg91qg2w8t35_pr0000gn/T/ipykernel_12395/2015311796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_enriched_movies_df.sort_values(by=['title', 'year', 'avg_rating', 'num_votes'], ascending=[True, False, False, False], ignore_index=True, inplace=True)


## Analysis of movies data:

#### *movie with maximum votes*

In [47]:
final_enriched_movies_df[final_enriched_movies_df['num_votes']==final_enriched_movies_df['num_votes'].max()]

,title,is_adult,language,year,runtime_mins,genre,avg_rating,rotten tomatoes,num_votes,streaming_on
594,interstellar,0,None,2014,169,"Adventure,Drama,Sci-Fi",8.6,None,1817999,data_unavailable


#### *movie with highest avg_rating*

In [48]:
final_enriched_movies_df[final_enriched_movies_df['avg_rating']==final_enriched_movies_df['avg_rating'].max()]

,title,is_adult,language,year,runtime_mins,genre,avg_rating,rotten tomatoes,num_votes,streaming_on
1431,the silence of swastika,0,None,2021,56,"Documentary,History",9.6,None,10280,data_unavailable


#### *Top 10 English movies with rating > 7*

In [58]:
final_enriched_movies_df[(final_enriched_movies_df['language'].notna()) & (final_enriched_movies_df['language'].str.contains('english', case=False)) & (final_enriched_movies_df['avg_rating'] > 7.0)].sort_values(by=['avg_rating'], ascending=False).head(10)

,title,is_adult,language,year,runtime_mins,genre,avg_rating,rotten tomatoes,num_votes,streaming_on
458,for sama,0,"Arabic,English",2019,100,"Biography,Documentary,War",8.5,99%,11763,Prime
945,parasite,0,"Korean,English",2019,132,"Drama,Thriller",8.5,99%,796776,Hulu
381,django unchained,0,"English,German,French,Italian",2012,165,"Drama,Western",8.4,87%,1547610,Netflix
631,joker,0,English,2019,122,"Crime,Drama,Thriller",8.4,,1274642,Netflix
308,coco,0,"English,Spanish",2017,105,"Adventure,Animation,Comedy",8.4,97%,506338,Disney+
144,avengers: endgame,0,"English,Japanese,Xhosa,German",2019,181,"Action,Adventure,Drama",8.4,93%,1123908,Disney+
145,avengers: infinity war,0,English,2018,149,"Action,Adventure,Sci-Fi",8.4,84%,1075329,Netflix
1141,spider-man: into the spider-verse,0,"English,Spanish",2018,117,"Action,Adventure,Animation",8.4,97%,526232,Netflix
1611,winter on fire: ukraine's fight for freedom,0,"Ukrainian,Russian,English",2015,102,Documentary,8.3,88%,19345,Netflix
591,inside out,0,English,2015,95,"Adventure,Animation,Comedy",8.2,25%,712609,Hulu


#### *count of Tamil movies by year which have avg_rating > 7*

In [69]:
final_enriched_movies_df[(final_enriched_movies_df['language'].str.contains('tamil', case=False)) & (final_enriched_movies_df['avg_rating'] > 7.0)]\
    .groupby(by='year')\
        .size()\
            .reset_index(name='counts')

,year,counts
0,2012,2
1,2014,3
2,2016,2
3,2017,4
4,2018,1
5,2019,1
6,2020,1
